## Brain Community Detection

##### DQM

In [ ]:
from QHyper.problems.brain_community_detection import BrainCommunityDetection


num_cases = 4
path = "../QHyper/problems/brain_community_data"
data_name = 'Edge_AAL90_Binary'
brain = BrainCommunityDetection(path, data_name, num_cases)
brain.objective_function

<class 'networkx.utils.decorators.argmap'> compilation 9:5: FutureWarning: modularity_matrix will return a numpy array instead of a matrix in NetworkX 3.0.


[]

In [ ]:
import sympy


G = brain.G

variables = sympy.symbols(' '.join([f'x{i}' for i in range(len(G.nodes))]))

In [ ]:
from sympy.core.expr import Expr
from typing import cast
from QHyper.hyperparameter_gen.parser import Expression


equation: Expr = cast(Expr, 0)
cross_out = [[0 for _ in range(len(G.nodes))] for _ in range(len(G.nodes))]
for i in G.nodes():
    for j in G.nodes():
        if i==j:
            continue
        if cross_out[i][j] != 0:
            continue
        u_var, v_var = variables[i], variables[j]
        print(u_var, v_var, i, j)
        equation += u_var*v_var*brain.B[i,j]
        cross_out[i][j] = 1
print("done")
equation *= -1
print("done")

# brain.objective_function = Expression(equation)

In [ ]:
equation_expression = Expression(equation)

In [ ]:
brain.objective_function = equation_expression

In [ ]:
from QHyper.solvers.dqm import DQM


brain_dqm = DQM(brain, 5)
# brain_sampleset = brain_dqm.solve()

In [ ]:
equation_expression.as_dict()


In [ ]:
brain_sampleset = brain_dqm.solve()

In [ ]:
brain_sampleset.first.sample

In [ ]:
import numpy as np


samples = brain_sampleset
sample = samples.first.sample
energy = samples.first.energy
run_time=(samples.info['run_time'])*0.001

communities=[]
for k in (range(brain.num_cases)):
    comm=[]
    for i in sample:
        if sample[i]==k:
            comm.append(i)
    communities.append(set(comm))

counts = np.zeros(brain.num_cases)
for s in sample.values():
    counts[s] += 1

print(sample)

In [ ]:
output_folder = "./demo_output"
results = communities, run_time, energy, counts, sample
brain.plot_results(output_folder, results)